In [0]:
-- Create Fact and Dimension tables.

DROP TABLE IF EXISTS dim_user;
CREATE TABLE dim_user as (select user_id from orders);

DROP TABLE IF EXISTS dim_products;
CREATE TABLE dim_products as (select product_id,product_name from products);

DROP TABLE IF EXISTS dim_department;
CREATE TABLE dim_department as (select department_id,department from departments);

DROP TABLE IF EXISTS dim_aisles;
CREATE TABLE dim_aisles as (select aisle_id,aisle from aisles);

DROP TABLE IF EXISTS dim_orders;
CREATE TABLE dim_orders as (select order_id,order_number,order_dow,order_hour_of_day, days_since_prior_order from orders);

DROP TABLE IF EXISTS fact_order_products;
CREATE TABLE fact_order_products as (select op.order_id,op.product_id,o.user_id,p.aisle_id,p.department_id,op.add_to_cart_order,op.reordered from order_products op join products p on op.product_id = p.product_id join orders o on op.order_id= o.order_id);

In [0]:
-- 1) Calculate the total number of products ordered per department:

SELECT
  d.department,
  COUNT(*) AS total_products_ordered
FROM
  fact_order_products fop
JOIN
  dim_department d ON fop.department_id = d.department_id
GROUP BY
  d.department;

In [0]:
-- 2) Find the top 5 aisles with the highest number of reordered products:

SELECT
  a.aisle,
  COUNT(*) AS total_reordered
FROM
  fact_order_products fop
JOIN
  dim_aisles a ON fop.aisle_id = a.aisle_id
WHERE
  fop.reordered = TRUE
GROUP BY
  a.aisle
ORDER BY
  total_reordered DESC
LIMIT 5;

In [0]:
-- 3) Identify the top 10 users with the highest number of unique products ordered:

SELECT
  u.user_id,
  COUNT(DISTINCT fop.product_id) AS unique_products_ordered
FROM
  fact_order_products fop
JOIN
  dim_user u ON fop.user_id = u.user_id
GROUP BY
  u.user_id
ORDER BY
  unique_products_ordered DESC
LIMIT 10;

In [0]:
-- 4) Calculate the average number of products added to the cart per order by day of the week:

SELECT
  o.order_dow,
  AVG(fop.add_to_cart_order) AS avg_products_per_order
FROM
  fact_order_products fop
JOIN
  dim_orders o ON fop.order_id = o.order_id
GROUP BY
  o.order_dow;